# Requirements

## main chute

Descent speed $ v < 9.14 m/s $ (or $ v < 30ft/s $)


## drogue chute

Descent speed $ 22.86 m/s - 45.72 m/s $ (or $ 75 ft/s - 150 ft/s $)




# Drag

Drag equation
$$
F_D = \frac{1}{2} \rho v^2 C_D A
$$

- $F_D$ drag force
- $\rho$ mass density of fluid
- $v$ velocity
- $A$ reference area
- $C_D$ drag coefficient

Descent velocity
$$
v = \sqrt{\frac{2 m g}{\rho_{air} C_D A}}
$$

In [1]:
import math

rho=1.2;
g=9.81;

def v_descent(m, cd, d):
    A = d**2*math.pi/4
    return math.sqrt(2*m*g/(rho*cd*A))

# Main chute
IFC-84: Iris Ultra 84 inch Standard Parachute (from fruitychutes.com)

- Cd: 2.20
- Diameter: 2.13m
- Type: Toroidal (Annular)
- Packing Volume: d=100mm, l=224mm, V=1.75l (assumes a very tight pack)
- Weight 0.54 kg
- Material: Nylon
- Number of Gores: 12

In [2]:
cd = 2.2; # drag coefficient
d = 2.13; # diameter [m]
v1 = v_descent(20, cd, d)
v2 = v_descent(16, cd, d)
print('20kg: {:.2f} m/s'.format(v1))
print('16kg: {:.2f} m/s (without 4kg payload)'.format(v2))

20kg: 6.46 m/s
16kg: 5.78 m/s (without 4kg payload)


# Drogue
CFC-18: 18 inch Elliptical Parachute (from fruitychutes.com)

- Cd between 1.5 - 1.6
- Diameter: 0.46m
- Weight: 49 grams
- Materials: 1.1oz Rip-stop, 220lb nylon shroud lines, 1000lb swivel
- Number of Gores: 8
- Packing Volume: d=48mm, l=84mm, V=0.15l (assumes a very tight pack)

In [3]:
cd = 1.6; # drag coefficient
d = 0.46; # diameter [m]
v1 = v_descent(20, cd, d)
v2 = v_descent(16, cd, d)
print('20kg: {:.2f} m/s'.format(v1))
print('16kg: {:.2f} m/s'.format(v2))

20kg: 35.07 m/s
16kg: 31.37 m/s
